In [1]:
import ollama
import chromadb

In [2]:
# Base chat comletion
chat_response: ollama.ChatResponse = ollama.chat(model='llama3.2:1b', messages=[
  {
    'role': 'user',
    'content': 'What is llama?',
  },
])
print(chat_response['message']['content'])

A llama is a domesticated mammal belonging to the camelid family. They are native to South America, specifically present in countries such as Peru, Bolivia, Ecuador, and Chile. Llamas have been used for various purposes, including:

1. Pack animals: Due to their strength, endurance, and ability to carry heavy loads, llamas were originally used as pack animals in the Andean highlands.
2. Livestock: Today, llamas are kept as livestock, providing meat, wool, and milk.
3. Companionship: Llamas are often raised on farms or ranches as companions for their owners, providing a unique relationship between humans and animals.

Llamas are known for several characteristics that distinguish them from other animals:

* Their distinctive "Brahma" sound: Llamas make a loud, trumpet-like noise, often used to communicate with each other.
* Their long necks and legs: Adult llamas typically have a height of around 5-6 feet (1.5-1.8 meters) at the shoulder, with a length of around 10-12 feet (3-4 meters).


In [3]:
# Add document to db
documents = [
  "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
  "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
  "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
  "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
  "Llamas are vegetarians and have very efficient digestive systems",
  "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
]

client = chromadb.Client()
collection = client.create_collection(name="docs")

# store each document in a vector embedding database
for i, d in enumerate(documents):
  response = ollama.embed(model="mxbai-embed-large", input=d)
  embeddings = response["embeddings"]
  collection.add(
    ids=[str(i)],
    embeddings=embeddings,
    documents=[d]
  )

In [4]:
# get single query embedding
input_prompt = "What animals are llamas related to?"

# generate an embedding for the input and retrieve the most relevant doc
response = ollama.embed(
  model="mxbai-embed-large",
  input=input_prompt
)

In [5]:
# retrieval by search method
retrieval_count_limit = 3
results = collection.query(
  query_embeddings=response["embeddings"],
  n_results=retrieval_count_limit
)
print(results['documents'])

data = results['documents'][0][0]

[["Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels", 'Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands', 'Llamas are vegetarians and have very efficient digestive systems']]


In [6]:
# generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model="llama3.2:1b",
  prompt=f"Using this data: {data}. Respond to this prompt: {input_prompt}"
)

print(output['response'])

Based on their classification, llamas are indeed part of the camelid family, which also includes:

1. Vicuñas (Vicugna pacos) - a South American species that's native to the Andes.
2. Alpacas (Vicugna americana) - another South American species closely related to llamas.
3. Camels (Camelus dromedarius and Camelus bactrianus) - although they're not as closely related to llamas as vicuñas and alpacas, camels are still part of the same family.

So, it's safe to say that llamas, vicuñas, and alpacas share a common ancestor in the camelid family tree.
